# MIHAC — Análisis Exploratorio del German Credit Dataset

**Notebook**: `01_eda_german_credit.ipynb`  
**Versión**: 1.0  
**Motor**: MIHAC v1.0 — Motor de Inferencia Heurística para Aprobación de Créditos  

---

## Objetivo

Explorar el German Credit Dataset (UCI, 1000 registros) ya **transformado por `mapper.py`**  
al formato MIHAC (9 variables de entrada + etiqueta real). Verificar distribuciones,  
correlaciones y la coherencia de los datos mapeados antes de usarlos como benchmark.

## Secciones

1. Imports y Carga de Datos
2. Variable Objetivo (Buenos vs Malos)
3. Análisis por Variable (8 subsecciones)
4. Correlaciones
5. Perfiles de Riesgo
6. Validación cruzada con MIHAC
7. Exportación de Figuras

In [ ]:
# ═══════════════════════════════════════════════════════════
# 1. IMPORTS Y CONFIGURACIÓN
# ═══════════════════════════════════════════════════════════

import sys
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de visualización
warnings.filterwarnings('ignore')
plt.rcParams.update({
    'figure.figsize': (12, 6),
    'figure.dpi': 100,
    'font.size': 11,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
})
sns.set_style('whitegrid')
sns.set_palette('Set2')

# Path del proyecto
PROJECT_ROOT = Path.cwd()
if PROJECT_ROOT.name == 'notebooks':
    PROJECT_ROOT = PROJECT_ROOT.parent
sys.path.insert(0, str(PROJECT_ROOT.parent))

print(f'Raíz del proyecto: {PROJECT_ROOT}')
print(f'Python: {sys.version}')

In [ ]:
# ═══════════════════════════════════════════════════════════
# 2. CARGA DE DATOS (mapper.py)
# ═══════════════════════════════════════════════════════════

from mihac.data.mapper import (
    GermanCreditMapper,
    get_dataset_summary,
    _generate_mock_raw,
)

mapper = GermanCreditMapper()

# Intentar cargar german.data, si no existe usar mock
german_path = PROJECT_ROOT / 'data' / 'german.data'
if german_path.exists():
    print('Cargando german.data real...')
    df = mapper.load_and_transform(str(german_path))
else:
    print('german.data no encontrado, intentando UCI...')
    try:
        df = mapper.load_and_transform(None)  # UCI
    except Exception:
        print('UCI no disponible, usando datos mock (100 registros)...')
        df_raw = _generate_mock_raw(100)
        df = mapper.transform(df_raw)

print(f'\nShape: {df.shape}')
print(f'Columnas: {df.columns.tolist()}')
df.head()

In [ ]:
# Resumen estadístico rápido
resumen = get_dataset_summary(df)
print('═' * 50)
print('RESUMEN DEL DATASET')
print('═' * 50)
for k, v in resumen.items():
    if isinstance(v, dict):
        print(f'  {k}:')
        for kk, vv in v.items():
            print(f'    {kk}: {vv}')
    else:
        print(f'  {k}: {v}')

print('\n── Descriptivas ──')
df.describe().round(2)

## 2. Variable Objetivo: Buenos vs Malos Pagadores

El German Credit Dataset tiene un desbalance 70/30:  
- **Clase 1**: Buen pagador (700 registros)  
- **Clase 2**: Mal pagador (300 registros)  

MIHAC **no es un clasificador ML** — es un sistema experto basado en reglas heurísticas.  
Sin embargo, comparar el dictamen de MIHAC contra la etiqueta real del dataset  
permite medir la *coherencia* del motor con datos históricos.

In [ ]:
# ═══════════════════════════════════════════════════════════
# 3. VARIABLE OBJETIVO
# ═══════════════════════════════════════════════════════════

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 3a. Barras de distribución
etiquetas = df['etiqueta_real'].value_counts().sort_index()
colores = ['#2ecc71', '#e74c3c']  # verde=bueno, rojo=malo
labels = ['Buen Pagador (1)', 'Mal Pagador (2)']

bars = axes[0].bar(labels, etiquetas.values, color=colores,
                   edgecolor='white', linewidth=1.5)
for bar, val in zip(bars, etiquetas.values):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 10,
                 f'{val}\n({val/len(df)*100:.1f}%)',
                 ha='center', va='bottom', fontweight='bold')
axes[0].set_title('Distribución de la Variable Objetivo')
axes[0].set_ylabel('Cantidad')
axes[0].set_ylim(0, max(etiquetas.values) * 1.2)

# 3b. Pie chart
axes[1].pie(etiquetas.values, labels=labels, colors=colores,
            autopct='%1.1f%%', startangle=90,
            explode=(0, 0.05), shadow=True,
            textprops={'fontsize': 12})
axes[1].set_title('Proporción Buenos vs Malos')

plt.suptitle('German Credit Dataset — Variable Objetivo',
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(str(PROJECT_ROOT / 'notebooks' / 'fig_objetivo.png'),
            bbox_inches='tight', dpi=150)
plt.show()

print(f'\nDesbalance: {etiquetas.iloc[0]/len(df)*100:.0f}%/{etiquetas.iloc[1]/len(df)*100:.0f}%')
print('NOTA: Este desbalance 70/30 es leve y no requiere oversampling para MIHAC.')

## 3. Análisis por Variable

Cada una de las 9 variables de entrada de MIHAC se analiza individualmente,  
mostrando su distribución global y segmentada por clase.

In [ ]:
# ═══════════════════════════════════════════════════════════
# 3.1 EDAD
# ═══════════════════════════════════════════════════════════

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histograma por clase
for clase, color, label in [(1, '#2ecc71', 'Bueno'),
                             (2, '#e74c3c', 'Malo')]:
    subset = df[df['etiqueta_real'] == clase]
    axes[0].hist(subset['edad'], bins=20, alpha=0.6,
                 color=color, label=label, edgecolor='white')

axes[0].set_title('Distribución de Edad por Clase')
axes[0].set_xlabel('Edad')
axes[0].set_ylabel('Frecuencia')
axes[0].legend()
axes[0].axvline(df['edad'].mean(), color='navy',
                linestyle='--', label=f'Media: {df["edad"].mean():.0f}')
axes[0].legend()

# Boxplot
df_plot = df.copy()
df_plot['Clase'] = df_plot['etiqueta_real'].map({1: 'Bueno', 2: 'Malo'})
sns.boxplot(x='Clase', y='edad', data=df_plot, ax=axes[1],
            palette={'Bueno': '#2ecc71', 'Malo': '#e74c3c'})
axes[1].set_title('Edad por Clase')

plt.suptitle('Variable: Edad', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f'Edad media global: {df["edad"].mean():.1f} años')
print(f'Edad media buenos: {df[df["etiqueta_real"]==1]["edad"].mean():.1f}')
print(f'Edad media malos:  {df[df["etiqueta_real"]==2]["edad"].mean():.1f}')

In [ ]:
# ═══════════════════════════════════════════════════════════
# 3.2 INGRESO MENSUAL (ESTIMADO)
# ═══════════════════════════════════════════════════════════

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for clase, color, label in [(1, '#2ecc71', 'Bueno'),
                             (2, '#e74c3c', 'Malo')]:
    subset = df[df['etiqueta_real'] == clase]
    axes[0].hist(subset['ingreso_mensual'], bins=30, alpha=0.6,
                 color=color, label=label, edgecolor='white')
axes[0].set_title('Distribución del Ingreso Mensual (estimado) por Clase')
axes[0].set_xlabel('Ingreso Mensual (MXN)')
axes[0].set_ylabel('Frecuencia')
axes[0].legend()

sns.boxplot(x='Clase', y='ingreso_mensual', data=df_plot, ax=axes[1],
            palette={'Bueno': '#2ecc71', 'Malo': '#e74c3c'})
axes[1].set_title('Ingreso Mensual por Clase')
axes[1].set_ylabel('Ingreso MXN')

plt.suptitle('Variable: Ingreso Mensual (estimado)', fontsize=14,
             fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f'Ingreso medio global:  ${df["ingreso_mensual"].mean():,.0f} MXN')
print(f'Ingreso medio buenos:  ${df[df["etiqueta_real"]==1]["ingreso_mensual"].mean():,.0f}')
print(f'Ingreso medio malos:   ${df[df["etiqueta_real"]==2]["ingreso_mensual"].mean():,.0f}')
print(f'\n⚠ NOTA: El ingreso es ESTIMADO desde A5/A2/A8 (ver docstring de _estimar_ingreso)')

In [ ]:
# ═══════════════════════════════════════════════════════════
# 3.3 TOTAL DEUDA ACTUAL (ESTIMADO)
# ═══════════════════════════════════════════════════════════

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for clase, color, label in [(1, '#2ecc71', 'Bueno'),
                             (2, '#e74c3c', 'Malo')]:
    subset = df[df['etiqueta_real'] == clase]
    axes[0].hist(subset['total_deuda_actual'], bins=30, alpha=0.6,
                 color=color, label=label, edgecolor='white')
axes[0].set_title('Distribución de Deuda Total por Clase')
axes[0].set_xlabel('Deuda Total (MXN)')
axes[0].set_ylabel('Frecuencia')
axes[0].legend()

sns.boxplot(x='Clase', y='total_deuda_actual', data=df_plot, ax=axes[1],
            palette={'Bueno': '#2ecc71', 'Malo': '#e74c3c'})
axes[1].set_title('Deuda Total por Clase')

plt.suptitle('Variable: Total Deuda Actual (estimada)', fontsize=14,
             fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# ═══════════════════════════════════════════════════════════
# 3.4 HISTORIAL CREDITICIO (0=Malo, 1=Neutro, 2=Bueno)
# ═══════════════════════════════════════════════════════════

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribución global
hist_labels = {0: 'Malo', 1: 'Neutro', 2: 'Bueno'}
hist_counts = df['historial_crediticio'].value_counts().sort_index()
bars = axes[0].bar([hist_labels[i] for i in hist_counts.index],
                   hist_counts.values,
                   color=['#e74c3c', '#f39c12', '#2ecc71'],
                   edgecolor='white')
for bar, val in zip(bars, hist_counts.values):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5,
                 str(val), ha='center', fontweight='bold')
axes[0].set_title('Distribución Global del Historial')
axes[0].set_ylabel('Cantidad')

# Historial por clase (stacked)
ct = pd.crosstab(df['historial_crediticio'], df['etiqueta_real'],
                 normalize='index')
ct.index = [hist_labels.get(i, str(i)) for i in ct.index]
ct.columns = ['Bueno', 'Malo']
ct.plot(kind='bar', stacked=True, ax=axes[1],
        color=['#2ecc71', '#e74c3c'], edgecolor='white')
axes[1].set_title('Tasa de Morosidad por Historial')
axes[1].set_ylabel('Proporción')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=0)
axes[1].legend(title='Clase')

plt.suptitle('Variable: Historial Crediticio', fontsize=14,
             fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# ═══════════════════════════════════════════════════════════
# 3.5 ANTIGÜEDAD LABORAL
# ═══════════════════════════════════════════════════════════

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

antig_counts = df['antiguedad_laboral'].value_counts().sort_index()
axes[0].bar(antig_counts.index, antig_counts.values,
            color='#3498db', edgecolor='white')
axes[0].set_title('Distribución de Antigüedad Laboral')
axes[0].set_xlabel('Años')
axes[0].set_ylabel('Cantidad')

sns.boxplot(x='Clase', y='antiguedad_laboral', data=df_plot, ax=axes[1],
            palette={'Bueno': '#2ecc71', 'Malo': '#e74c3c'})
axes[1].set_title('Antigüedad por Clase')

plt.suptitle('Variable: Antigüedad Laboral', fontsize=14,
             fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# ═══════════════════════════════════════════════════════════
# 3.6 TIPO DE VIVIENDA
# ═══════════════════════════════════════════════════════════

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

viv_counts = df['tipo_vivienda'].value_counts()
colores_viv = {'Propia': '#2ecc71', 'Familiar': '#3498db', 'Rentada': '#e74c3c'}
bars = axes[0].bar(viv_counts.index, viv_counts.values,
                   color=[colores_viv.get(v, '#999') for v in viv_counts.index],
                   edgecolor='white')
for bar, val in zip(bars, viv_counts.values):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5,
                 f'{val} ({val/len(df)*100:.1f}%)',
                 ha='center', fontweight='bold')
axes[0].set_title('Distribución de Tipo de Vivienda')
axes[0].set_ylabel('Cantidad')

# Tasa de morosidad por vivienda
ct = pd.crosstab(df['tipo_vivienda'], df['etiqueta_real'],
                 normalize='index')
ct.columns = ['Bueno', 'Malo']
ct.plot(kind='bar', stacked=True, ax=axes[1],
        color=['#2ecc71', '#e74c3c'], edgecolor='white')
axes[1].set_title('Tasa de Morosidad por Vivienda')
axes[1].set_ylabel('Proporción')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=0)

plt.suptitle('Variable: Tipo de Vivienda', fontsize=14,
             fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# ═══════════════════════════════════════════════════════════
# 3.7 PROPÓSITO DEL CRÉDITO
# ═══════════════════════════════════════════════════════════

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

prop_counts = df['proposito_credito'].value_counts()
colores_prop = {
    'Negocio': '#27ae60', 'Educacion': '#2980b9',
    'Consumo': '#f39c12', 'Emergencia': '#e67e22',
    'Vacaciones': '#e74c3c'
}
bars = axes[0].bar(prop_counts.index, prop_counts.values,
                   color=[colores_prop.get(p, '#999') for p in prop_counts.index],
                   edgecolor='white')
for bar, val in zip(bars, prop_counts.values):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5,
                 str(val), ha='center', fontweight='bold', fontsize=9)
axes[0].set_title('Distribución por Propósito')
axes[0].set_ylabel('Cantidad')
axes[0].tick_params(axis='x', rotation=30)

# Tasa de morosidad por propósito
ct = pd.crosstab(df['proposito_credito'], df['etiqueta_real'],
                 normalize='index')
ct.columns = ['Bueno', 'Malo']
ct.plot(kind='barh', stacked=True, ax=axes[1],
        color=['#2ecc71', '#e74c3c'], edgecolor='white')
axes[1].set_title('Tasa de Morosidad por Propósito')
axes[1].set_xlabel('Proporción')

plt.suptitle('Variable: Propósito del Crédito', fontsize=14,
             fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# ═══════════════════════════════════════════════════════════
# 3.8 MONTO DE CRÉDITO
# ═══════════════════════════════════════════════════════════

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for clase, color, label in [(1, '#2ecc71', 'Bueno'),
                             (2, '#e74c3c', 'Malo')]:
    subset = df[df['etiqueta_real'] == clase]
    axes[0].hist(subset['monto_credito'], bins=30, alpha=0.6,
                 color=color, label=label, edgecolor='white')
axes[0].set_title('Distribución del Monto de Crédito por Clase')
axes[0].set_xlabel('Monto (MXN)')
axes[0].set_ylabel('Frecuencia')
axes[0].legend()
axes[0].axvline(20000, color='red', linestyle='--',
                label='Umbral MIHAC ($20K → 85pts)')
axes[0].legend()

sns.boxplot(x='Clase', y='monto_credito', data=df_plot, ax=axes[1],
            palette={'Bueno': '#2ecc71', 'Malo': '#e74c3c'})
axes[1].set_title('Monto por Clase')
axes[1].axhline(20000, color='red', linestyle='--', alpha=0.7)

plt.suptitle('Variable: Monto de Crédito (MXN)', fontsize=14,
             fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

pct_alto = (df['monto_credito'] > 20000).sum() / len(df) * 100
print(f'Montos > $20,000 (umbral 85): {pct_alto:.1f}% del dataset')

## 4. DTI Estimado y Correlaciones

In [ ]:
# ═══════════════════════════════════════════════════════════
# 4. DTI Y CORRELACIONES
# ═══════════════════════════════════════════════════════════

# Calcular DTI estimado
df['dti_estimado'] = df['total_deuda_actual'] / df['ingreso_mensual']

# Clasificar DTI según umbrales MIHAC
def clasificar_dti(dti):
    if dti < 0.25:
        return 'BAJO'
    elif dti < 0.40:
        return 'MODERADO'
    elif dti < 0.60:
        return 'ALTO'
    else:
        return 'CRITICO'

df['dti_clase'] = df['dti_estimado'].apply(clasificar_dti)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 4a. Distribución DTI
for clase, color, label in [(1, '#2ecc71', 'Bueno'),
                             (2, '#e74c3c', 'Malo')]:
    subset = df[df['etiqueta_real'] == clase]
    axes[0].hist(subset['dti_estimado'], bins=30, alpha=0.6,
                 color=color, label=label, edgecolor='white')
# Líneas de umbrales
for umbral, label_u in [(0.25, 'BAJO'), (0.40, 'MOD'),
                         (0.60, 'ALTO/CRIT')]:
    axes[0].axvline(umbral, color='gray', linestyle='--', alpha=0.7)
    axes[0].text(umbral+0.01, axes[0].get_ylim()[1]*0.9,
                 label_u, fontsize=8, color='gray')
axes[0].set_title('Distribución DTI Estimado')
axes[0].set_xlabel('DTI')
axes[0].legend()

# 4b. DTI por clase MIHAC
dti_order = ['BAJO', 'MODERADO', 'ALTO', 'CRITICO']
dti_colores = {'BAJO': '#2ecc71', 'MODERADO': '#f39c12',
               'ALTO': '#e67e22', 'CRITICO': '#e74c3c'}
dti_counts = df['dti_clase'].value_counts().reindex(dti_order, fill_value=0)
bars = axes[1].bar(dti_counts.index, dti_counts.values,
                   color=[dti_colores[d] for d in dti_counts.index],
                   edgecolor='white')
for bar, val in zip(bars, dti_counts.values):
    if val > 0:
        axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 3,
                     str(val), ha='center', fontweight='bold')
axes[1].set_title('Clasificación DTI (umbrales MIHAC)')
axes[1].set_ylabel('Cantidad')

# 4c. Heatmap de correlaciones
cols_num = ['edad', 'ingreso_mensual', 'total_deuda_actual',
            'antiguedad_laboral', 'numero_dependientes',
            'monto_credito', 'historial_crediticio',
            'dti_estimado', 'etiqueta_binaria']
corr = df[cols_num].corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, annot=True, fmt='.2f',
            cmap='RdBu_r', center=0, ax=axes[2],
            square=True, linewidths=0.5,
            cbar_kws={'shrink': 0.8})
axes[2].set_title('Matriz de Correlación')

plt.suptitle('DTI Estimado y Correlaciones', fontsize=14,
             fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(str(PROJECT_ROOT / 'notebooks' / 'fig_correlaciones.png'),
            bbox_inches='tight', dpi=150)
plt.show()

print('\n── Correlaciones con etiqueta_binaria ──')
print(corr['etiqueta_binaria'].sort_values().to_string())

## 5. Perfiles de Riesgo

Segmentación multi-variable para identificar patrones de riesgo naturales en el dataset.

In [ ]:
# ═══════════════════════════════════════════════════════════
# 5. PERFILES DE RIESGO
# ═══════════════════════════════════════════════════════════

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# 5a. Scatter: Ingreso vs DTI, coloreado por clase
for clase, color, label in [(1, '#2ecc71', 'Bueno'),
                             (2, '#e74c3c', 'Malo')]:
    subset = df[df['etiqueta_real'] == clase]
    axes[0].scatter(subset['ingreso_mensual'], subset['dti_estimado'],
                    alpha=0.4, color=color, label=label, s=20)
axes[0].axhline(0.60, color='red', linestyle='--', alpha=0.8,
                label='DTI CRITICO (0.60)')
axes[0].axhline(0.25, color='green', linestyle='--', alpha=0.5,
                label='DTI BAJO (0.25)')
axes[0].set_xlabel('Ingreso Mensual (MXN)')
axes[0].set_ylabel('DTI Estimado')
axes[0].set_title('Ingreso vs DTI por Clase')
axes[0].legend(fontsize=9)

# 5b. Pairplot simplificado: edad vs monto, por clase
for clase, color, label in [(1, '#2ecc71', 'Bueno'),
                             (2, '#e74c3c', 'Malo')]:
    subset = df[df['etiqueta_real'] == clase]
    axes[1].scatter(subset['edad'], subset['monto_credito'],
                    alpha=0.4, color=color, label=label, s=20)
axes[1].axhline(20000, color='orange', linestyle='--', alpha=0.7,
                label='Monto alto ($20K)')
axes[1].set_xlabel('Edad')
axes[1].set_ylabel('Monto Crédito (MXN)')
axes[1].set_title('Edad vs Monto por Clase')
axes[1].legend(fontsize=9)

plt.suptitle('Perfiles de Riesgo', fontsize=14,
             fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(str(PROJECT_ROOT / 'notebooks' / 'fig_perfiles_riesgo.png'),
            bbox_inches='tight', dpi=150)
plt.show()

# Tabla resumen por DTI
print('\n── Tasa de morosidad por DTI ──')
for dti_cls in dti_order:
    sub = df[df['dti_clase'] == dti_cls]
    if len(sub) > 0:
        tasa = (sub['etiqueta_real'] == 2).sum() / len(sub) * 100
        print(f'  {dti_cls:<10}: {len(sub):>4} registros, '
              f'morosidad {tasa:.1f}%')

## 6. Validación Cruzada con MIHAC

Evaluamos los 1000 registros transformados con el InferenceEngine y comparamos  
el dictamen MIHAC contra la etiqueta real del dataset.  

**IMPORTANTE**: MIHAC es un sistema experto basado en reglas, NO un clasificador ML.  
No se espera accuracy > 90%. La coherencia se mide observando si la tendencia  
general del motor es sensata.

In [ ]:
# ═══════════════════════════════════════════════════════════
# 6. VALIDACIÓN CON MIHAC
# ═══════════════════════════════════════════════════════════

from mihac.core.engine import InferenceEngine

engine = InferenceEngine()

# Convertir a dicts y evaluar
dicts = mapper.to_mihac_dicts(df)
print(f'Evaluando {len(dicts)} solicitudes con MIHAC...')
resultados = engine.evaluate_batch(dicts)
print(f'Completado: {engine.stats["evaluaciones_por_segundo"]:.0f} eval/seg')

# Agregar resultados al DataFrame
df['mihac_score'] = [r['score_final'] for r in resultados]
df['mihac_dictamen'] = [r['dictamen'] for r in resultados]
df['mihac_dti_pct'] = [
    float(r['dti']['valor_porcentaje'].replace('%', '')) / 100
    for r in resultados
]

print(f'\n── Distribución de dictámenes MIHAC ──')
for d in ['APROBADO', 'REVISION_MANUAL', 'RECHAZADO']:
    cnt = (df['mihac_dictamen'] == d).sum()
    print(f'  {d:<18}: {cnt:>4} ({cnt/len(df)*100:.1f}%)')

In [ ]:
# ═══════════════════════════════════════════════════════════
# 6b. VISUALIZACIÓN: MIHAC vs ETIQUETA REAL
# ═══════════════════════════════════════════════════════════

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 6b-1. Distribución de scores por clase real
for clase, color, label in [(1, '#2ecc71', 'Bueno (real)'),
                             (2, '#e74c3c', 'Malo (real)')]:
    subset = df[df['etiqueta_real'] == clase]
    axes[0].hist(subset['mihac_score'], bins=20, alpha=0.6,
                 color=color, label=label, edgecolor='white')
axes[0].axvline(80, color='green', linestyle='--', label='Umbral 80')
axes[0].axvline(60, color='orange', linestyle='--', label='Umbral 60')
axes[0].set_title('Score MIHAC por Clase Real')
axes[0].set_xlabel('Score MIHAC')
axes[0].legend(fontsize=9)

# 6b-2. Tabla cruzada: dictamen × clase real
ct = pd.crosstab(df['mihac_dictamen'], df['etiqueta_real'])
ct.columns = ['Bueno', 'Malo']
ct = ct.reindex(['APROBADO', 'REVISION_MANUAL', 'RECHAZADO'])
ct.plot(kind='bar', ax=axes[1],
        color=['#2ecc71', '#e74c3c'], edgecolor='white')
axes[1].set_title('Dictamen MIHAC × Clase Real')
axes[1].set_ylabel('Cantidad')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=30)
axes[1].legend(title='Clase Real')

# 6b-3. Score promedio por clase
score_por_clase = df.groupby('etiqueta_real')['mihac_score'].mean()
bars = axes[2].bar(['Bueno (1)', 'Malo (2)'],
                   score_por_clase.values,
                   color=['#2ecc71', '#e74c3c'],
                   edgecolor='white')
for bar, val in zip(bars, score_por_clase.values):
    axes[2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                 f'{val:.1f}', ha='center', fontweight='bold')
axes[2].set_title('Score MIHAC Promedio por Clase Real')
axes[2].set_ylabel('Score Promedio')
axes[2].axhline(80, color='green', linestyle='--', alpha=0.5)

plt.suptitle('Validación: MIHAC vs Etiqueta Real', fontsize=14,
             fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(str(PROJECT_ROOT / 'notebooks' / 'fig_validacion_mihac.png'),
            bbox_inches='tight', dpi=150)
plt.show()

# Métricas de coherencia
print('\n── Métricas Básicas de Coherencia ──')
print(f'Score promedio buenos: {score_por_clase.get(1, 0):.1f}/100')
print(f'Score promedio malos:  {score_por_clase.get(2, 0):.1f}/100')
print(f'Diferencia:           {abs(score_por_clase.get(1,0) - score_por_clase.get(2,0)):.1f} puntos')
print()
if score_por_clase.get(1, 0) > score_por_clase.get(2, 0):
    print('✓ Coherencia: Buenos pagadores obtienen scores más altos que malos.')
else:
    print('⚠ Incoherencia: Revisar calibración de reglas.')

# Tabla cruzada detallada
print('\n── Tabla Cruzada: Dictamen × Clase Real ──')
print(ct.to_string())

## 7. Exportación de Figuras

Las figuras generadas se guardaron automáticamente en `notebooks/`.  
Usar para el capítulo de Resultados de la tesis.

In [ ]:
# ═══════════════════════════════════════════════════════════
# 7. RESUMEN Y EXPORTACIÓN
# ═══════════════════════════════════════════════════════════

# Exportar DataFrame final
output_path = PROJECT_ROOT / 'data' / 'german_credit_mihac.csv'
df.to_csv(output_path, index=False, encoding='utf-8')
print(f'Dataset transformado exportado a: {output_path}')

# Resumen final
print(f'\n{"═"*60}')
print('RESUMEN DEL EDA')
print(f'{"═"*60}')
print(f'Total registros: {len(df)}')
print(f'Variables de entrada: 9')
print(f'Buenos pagadores: {(df["etiqueta_real"]==1).sum()}')
print(f'Malos pagadores:  {(df["etiqueta_real"]==2).sum()}')
print(f'Score MIHAC promedio: {df["mihac_score"].mean():.1f}')
print(f'\nArchivos generados:')

import os
notebooks_dir = PROJECT_ROOT / 'notebooks'
for f in sorted(notebooks_dir.glob('fig_*.png')):
    size_kb = f.stat().st_size / 1024
    print(f'  {f.name} ({size_kb:.0f} KB)')

print(f'\n  {output_path.name}')
print(f'\n{"═"*60}')
print('EDA COMPLETADO ✓')
print(f'{"═"*60}')